In [1]:
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from time import sleep
import datetime
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')

import Alpha as alpha

# 隔夜聪明钱

In [9]:
data = os.listdir(r"D:\实习\研报复现\dataAll\stockMinData\volume")
res = []

for date in tqdm(data):
    volume_path = os.path.join(r"D:\实习\研报复现\dataAll\stockMinData\volume",date)
    date = pd.to_datetime(date.split('.')[0],format='%Y%m%d')
    volume = pd.read_parquet(volume_path).iloc[0]
    volume.name = date
    
    res.append(volume)
intraday = pd.concat(res,axis=1).T

100%|██████████████████████████████████████████████████████████████████████████████| 2512/2512 [09:48<00:00,  4.27it/s]


In [24]:
close = alpha.load_data('pv\\stock_close')
opn = alpha.load_data('pv\\stock_open')
volume = alpha.load_data('pv\\stock_volume')
shareout = alpha.load_data('pv\\stock_shareout')

overnight_tnv = intraday.div(shareout,axis=0).dropna(axis=0,how='all').dropna(axis=0,how='all')

overnight = (opn.div(close.shift(1),axis=0) - 1).dropna(axis=0,how='all')
overnight_smart = ((overnight - overnight.rolling(20,min_periods=1).min())/
                   (overnight.rolling(20,min_periods=1).max()-overnight.rolling(20,min_periods=1).min())).iloc[19:]

overnight_smart20 = overnight_smart.div(overnight_tnv,axis=0).dropna(axis=0,how='all').rolling(20,min_periods=1).mean().iloc[19:]

In [27]:
alpha.Alpha(overnight_smart20, 'overnight_smart20', verbose=True, freq='M', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\alpha_result\report\overnight_smart20_M.xlsx


# 日内换手率

In [50]:
CTR = []
intraday_tnv = ((volume-intraday).div(shareout,axis=0)).dropna(axis=0,how='all').shift(1).dropna(axis=0,how='all').loc['2014-01-29':]
overnight_smart_20 = overnight_smart20.loc[:'2024-03-29']

idx = overnight_smart_20.columns.intersection(intraday_tnv.columns)

for dayID in tqdm(range(20,len(overnight_smart_20)+1)):
    overnight_smart_20_ = overnight_smart20.iloc[dayID-20:dayID].rank(pct=False)[idx]  
    turnover_20 = intraday_tnv.iloc[dayID-20:dayID][idx]
    
    condition1 = overnight_smart_20_<=4
    
    # 使用numpy.select选择对应位置的B表的值
    selected_values1 = np.select([condition1], [turnover_20],default=np.nan)

    # 创建一个新的dataframe来存储这些值
    value1 = pd.DataFrame(selected_values1, index=turnover_20.index, columns=turnover_20.columns)

    # 计算均值
    mean_values1 = value1.mean()
    
    CTR.append(mean_values1)

CTR = pd.concat(CTR,axis=1).T
CTR.index = intraday_tnv.index[19:]

# CTRNeu = alpha.mvind_neutral(CTR)

100%|██████████████████████████████████████████████████████████████████████████████| 2454/2454 [05:43<00:00,  7.14it/s]


In [59]:
import statsmodels.api as sm
mv = np.log(alpha.load_data("pv\\stock_mv")).stack()
res = []
for idx in tqdm(CTR.index):
    temp = pd.concat([CTR.loc[idx], mv.loc[idx]], axis=1).dropna()

    X = temp.iloc[:, 1:]
    X = sm.add_constant(X)
    y = temp.iloc[:, 0]

    # 创建模型
    model = sm.OLS(y, X)

    # 拟合模型
    results = model.fit()

    res.append(results.resid)

fctNeu = pd.concat(res, axis=1).T
fctNeu.index = CTR.index

100%|█████████████████████████████████████████████████████████████████████████████| 2454/2454 [00:15<00:00, 156.13it/s]


In [4]:
CTR = alpha.load_fct('CTRNeu',fct_path='D:\\实习\\研报复现\\QuantResearch\\pv_alpha')

In [5]:
alpha.Alpha(CTR, 'CTRNeu', verbose=True, freq='M', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\alpha_result\report\CTRNeu_M.xlsx


In [62]:
alpha.Alpha(fctNeu, 'CTRNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\alpha_result\report\CTRNeu_W.xlsx


In [63]:
alpha.to_fctDataBase(fctNeu.dropna(axis=0,how='all'),'CTRNeu')